In [1]:
!pip install julep

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.3/271.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.1/754.1 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.1.1
    Uninstalling python-dotenv-1.1.1:
      Successfully uninstalled python-dotenv-1.1.1


In [84]:
import yaml
with open("config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)
API_KEY = config['julep']['api_key']

In [86]:
import yaml
import os
from julep import Client

# --- create client ---
client = Client(api_key=API_KEY, environment="production")
project_name = 'hiringAssistant___'
# --- create a project ---
project = client.projects.create(
    name="Hiring Assistant",
    # optional fields:
    canonical_name=project_name,
    metadata={"owner": "recruiting", "env": "dev"}
)

print("Project created:")
print("  id:            ", project.id)
print("  name:          ", project.name)
print("  canonical_name:", project.canonical_name)
print("  created_at:    ", project.created_at)
print("  updated_at:    ", project.updated_at)

# (optional) list projects to verify
projects = client.projects.list(limit=10)

Project created:
  id:             068ab746-9f30-7e0f-8000-90e6b80b4713
  name:           Hiring Assistant
  canonical_name: hiringAssistant___
  created_at:     2025-08-24 20:22:01.953262+00:00
  updated_at:     2025-08-24 20:22:01.953262+00:00


In [91]:
# --- Cell 1: Setup & Agents ---
import os, time, json, uuid, yaml
from julep import Julep

# Load API key
with open("config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)
API_KEY = config["julep"]["api_key"]

client = Julep(api_key=API_KEY)

def safe_json_loads(s):
    try:
        return json.loads(s)
    except Exception:
        return None

def run_and_wait(task_id, task_input):
    exe = client.executions.create(task_id=task_id, input=task_input)
    print("Execution:", exe.id)
    while True:
        exe = client.executions.get(exe.id)
        print("Status:", exe.status)
        if exe.status in ("succeeded", "failed", "cancelled"):
            break
        time.sleep(1)
    print("Final status:", exe.status)
    return exe

# ---------- Agents (temperature 0.0 for determinism) ----------

# ExtractorAgent — one profile per resume, verbatim-only
extractor = client.agents.create(
    name="ExtractorAgent1",
    about="Extract structured evidence from a single resume: skills, experience, education, projects; strictly from provided text.",
    instructions="Return ONLY valid JSON. No prose or code fences.",
    project=project_name,
    default_settings={
        "temperature": 0.0,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "ExtractedProfile",
                "schema": {
                    "type": "object",
                    "properties": {
                        "candidate_id": {"type": "string"},
                        "name": {"type": "string"},
                        "skills": {"type": "array", "items": {"type": "string"}},
                        "experience": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "title": {"type": "string"},
                                    "company": {"type": "string"},
                                    "years": {"type": "number"},
                                    "highlights": {"type": "array", "items": {"type": "string"}}
                                }
                            }
                        },
                        "education": {"type": "array", "items": {"type": "string"}},
                        "projects": {"type": "array", "items": {"type": "string"}},
                        "flags": {
                            "type": "object",
                            "properties": {
                                "disqualifiers_hit": {"type": "array", "items": {"type": "string"}}
                            }
                        }
                    },
                    # "required": ["candidate_id", "name", "skills", "experience", "education", "projects", "flags"]
                    "required": ["extracted_profiles"]
                }
            }
        }
    }
)
print("ExtractorAgent:", extractor.id)

# ScorerRankerAgent
scorer = client.agents.create(
    name="ScorerRankerAgent1",
    about="Score & rank candidates against criteria (weights, disqualifiers). Deterministic & auditable.",
    instructions="Return ONLY valid JSON. No prose or code fences.",
    project=project_name,
    default_settings={
        "temperature": 0.0,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "ScoreRank",
                "schema": {
                    "type": "object",
                    "properties": {
                        "ranked": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "candidate_id": {"type": "string"},
                                    "name": {"type": "string"},
                                    "score_total": {"type": "number"},
                                    "breakdown": {
                                        "type": "object",
                                        "properties": {
                                            "must_haves": {
                                                "type": "object",
                                                "properties": {
                                                    "matched": {"type": "array", "items": {"type": "string"}},
                                                    "missing": {"type": "array", "items": {"type": "string"}},
                                                    "score": {"type": "number"}
                                                }
                                            },
                                            "nice_to_haves": {
                                                "type": "object",
                                                "properties": {
                                                    "matched": {"type": "array", "items": {"type": "string"}},
                                                    "score": {"type": "number"}
                                                }
                                            },
                                            "experience": {"type": "number"},
                                            "education": {"type": "number"},
                                            "projects": {"type": "number"},
                                            "penalties": {
                                                "type": "object",
                                                "properties": {
                                                    "disqualifiers": {"type": "array", "items": {"type": "string"}},
                                                    "score_delta": {"type": "number"}
                                                }
                                            }
                                        }
                                    },
                                    "rationale": {"type": "string"}
                                },
                                "required": ["candidate_id", "name", "score_total", "breakdown", "rationale"]
                            }
                        },
                        "top_n_ids": {"type": "array", "items": {"type": "string"}}
                    },
                    "required": ["ranked", "top_n_ids"]
                }
            }
        }
    }
)
print("ScorerRankerAgent:", scorer.id)

# InterviewerAgent
interviewer = client.agents.create(
    name="InterviewerAgent1",
    about="Write tailored interview questions referencing candidate background and must-have skills.",
    instructions="Return ONLY valid JSON. No prose or code fences.",
    project=project_name,
    default_settings={
        "temperature": 0.0,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "InterviewQuestions",
                "schema": {
                    "type": "object",
                    "properties": {
                        "interview_questions": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "candidate_id": {"type": "string"},
                                    "name": {"type": "string"},
                                    "questions": {
                                        "type": "array",
                                        "items": {
                                            "type": "object",
                                            "properties": {
                                                "question": {"type": "string"},
                                                "why_this_question": {"type": "string"}
                                            },
                                            "required": ["question"]
                                        },
                                        "minItems": 6,
                                        "maxItems": 10
                                    }
                                },
                                "required": ["candidate_id", "name", "questions"]
                            }
                        }
                    },
                    "required": ["interview_questions"]
                }
            }
        }
    }
)
print("InterviewerAgent:", interviewer.id)

# ---------- Sample inputs (Artist role) ----------
# criteria = {
#     "role": "An Artist",
#     "must_haves": ["Dancing", "Acting", "Singing"],
#     "nice_to_haves": ["2y experience in acting"],
#     "weights": {
#         "must_haves": 0.5, "nice_to_haves": 0.2, "experience": 0.2, "projects": 0.1,
#         "disqualifier_penalty": -1.0
#     },
#     "disqualifiers": ["No work authorization", "Under 2 years experience"]
# }

# resumes = [
#     {"candidate_id": "c1", "name": "Nour Nou",  "text": "Dancing, Acting, Singing; 5y Acting; professional artist; theater and stage performances."},
#     {"candidate_id": "c2", "name": "Layla Zee", "text": "Contemporary Dancing, Singing; 3y stage musicals; community theater actress; vocal ensemble projects."},
#     {"candidate_id": "c3", "name": "Omar Ray",  "text": "Acting, voice-over; 6y drama roles; beginner singing lessons; short-film projects."},
# ]

criteria = {
    "role": "Senior Backend Engineer",
    "must_haves": ["Python", "PostgreSQL", "Distributed systems"],
    "nice_to_haves": ["Kubernetes", "AWS", "gRPC"],
    "weights": {
        "must_haves": 0.5,
        "nice_to_haves": 0.2,
        "experience": 0.2,
        "projects": 0.1,
        "disqualifier_penalty": -1.0
    },
    "disqualifiers": ["No work authorization", "Under 2 years experience"]
}

resumes = [
    {"candidate_id": "c1", "name": "Alice Johnson", "text": "Senior Backend Engineer at Stripe, Python, Go, PostgreSQL, Kubernete, B.Sc. in Computer Science from University of Tech"},
    {"candidate_id": "c2", "name": "Bob Smith", "text": "Data Scientist at Spotify TensorFlow, PyTorch, Spark"},
    {"candidate_id": "c3", "name": "Clara Evans", "text": "Frontend React/TS engineer at Airbnb"},
    {"candidate_id": "c4", "name": "Daniel Kim", "text": "Cloud Architect at AWS Kubernetes, Terraform, Prometheus"},
    {"candidate_id": "c5", "name": "Elena Petrova", "text": "AI Researcher (Transformers, JAX)."},
    {"candidate_id": "c6", "name": "Farid Al-Mansouri", "text": "Senior iOS Dev (Swift, Kotlin, Flutter)"},
    {"candidate_id": "c7", "name": "Grace Liu", "text": "Cybersecurity Analyst (IDS/IPS, SIEM)"},
    {"candidate_id": "c8", "name": "Hassan Omar", "text": "Technical PM (Agile, Jira)"},
    {"candidate_id": "c9", "name": "Isabella Rossi", "text": "UX Designer (Figma, Research, Accessibility."},
    {"candidate_id": "c10","name": "Jamal Wright", "text":"Robotics Engineer (C++, ROS, SLAM, CV)"},
]
N = 3


ExtractorAgent: 068ab750-f292-76cf-8000-7fbe36f0f5b3
ScorerRankerAgent: 068ab750-f6bb-752e-8000-6c46f305f47c
InterviewerAgent: 068ab750-f9cd-7574-8000-badf30a72998


In [92]:
# --- Cell 2: Task A — Extract Evidence (verbatim-only, role-aware context) ---

task_extract_def = {
    "name": "extract_task",
    "description": "Extract structured evidence from resumes with strict verbatim matching and 1:1 mapping.",
    "input_schema": {
        "type": "object",
        "required": ["criteria", "resumes"],
        "properties": {
            "criteria": {"type": "object"},
            "resumes": {
                "type": "array",
                "items": {"type": "object", "required": ["candidate_id", "name", "text"]}
            }
        }
    },
    "main": [
        {
            "prompt": [
                {
                    "role": "system",
                    "content": (
                        "You are ExtractorAgent that takes from resumes and fill in the corresponding field.\n"
                        "Role context: You will receive a role and must-haves, but you must EXTRACT ONLY what appears in each resume's `text`. Do not hallucinate resumes.\n"
                        "Hard rules:\n"
                        "1) Output exactly one profile per input resume, in the SAME ORDER. Do not add or drop candidates.\n"
                        "2) Copy 'candidate_id' and 'name' EXACTLY from the input for each profile.\n"
                        "3) 'skills', 'experience', 'education', 'projects' must be derived ONLY from resume.text; if not present, use [].\n"
                        "4) If a must-have isn't present verbatim in the text, leave it missing—do NOT add it.\n"
                        "5) flags.disqualifiers_hit can be left [] unless a disqualifier is explicitly stated.\n\n"
                        "Return ONLY valid JSON with key 'extracted_profiles' (array of profiles). JSON only; no prose."
                    )
                },
                {
                    "role": "user",
                    "content": "$ f'''Role criteria (context only, do NOT invent): {{ steps[0].input.criteria | tojson }}\n\nResumes (extract strictly from `text` and preserve order): {{ steps[0].input.resumes | tojson }}'''"
                },
            ],
            "unwrap": True,
            "save_as": "extracted_json",  # JSON string
        },
        {"return": {"extracted_json": "$ steps[0].output"}},
    ],
}

extract_task = client.tasks.create(agent_id=extractor.id, **task_extract_def)
print("Task A ready:", extract_task.id, extract_task.name)

# ---------- Execution A ----------
exe_a = run_and_wait(extract_task.id, {"criteria": criteria, "resumes": resumes})
if exe_a.status != "succeeded":
    raise RuntimeError(f"Task A failed: {exe_a.output}")

out_a = exe_a.output
parsed = safe_json_loads(out_a) if isinstance(out_a, str) else (out_a or {})
extracted_json = parsed.get("extracted_json", parsed)

# Parse model JSON to Python
extracted_parsed = safe_json_loads(extracted_json) if isinstance(extracted_json, str) else (extracted_json or {})
extracted_profiles = extracted_parsed.get("extracted_profiles", [])

print("Extracted profiles:", json.dumps(extracted_profiles, indent=3, ensure_ascii=False))


Task A ready: 068ab751-1fd7-7b66-8000-11bbd1c0b869 extract_task
Execution: 068ab751-2540-7e85-8000-5944e7d66640
Status: queued
Status: starting
Status: starting
Status: starting
Status: succeeded
Final status: succeeded
Extracted profiles: [
   {
      "candidate_id": "1",
      "name": "Alice Johnson",
      "skills": [
         "Python",
         "Java",
         "SQL"
      ],
      "experience": [
         {
            "title": "Software Engineer",
            "years": 3,
            "company": "Tech Solutions",
            "highlights": [
               "Developed web applications using Python and Java",
               "Collaborated with cross-functional teams to design scalable solutions"
            ]
         }
      ],
      "education": [
         "B.Sc. in Computer Science from University of Example"
      ],
      "projects": [
         "Automated testing framework for web applications"
      ],
      "flags": {
         "disqualifiers_hit": []
      }
   },
   {
      "ca

In [93]:
# --- Cell 3: Task B — Score & Rank (create & execute) ---

task_score_def = {
    "name": "score_rank_task",
    "description": "Score candidates against criteria, enforce disqualifiers, and rank.",
    "input_schema": {
        "type": "object",
        "required": ["criteria", "extracted_profiles", "N"],
        "properties": {
            "criteria": {"type": "object"},
            "extracted_profiles": {"type": "array", "items": {"type": "object"}},
            "N": {"type": "integer", "minimum": 1}
        }
    },
    "main": [
        {
            "prompt": [
                {
                    "role": "system",
                    "content": (
                        "You are ScorerRankerAgent.\n"
                        "Score ONLY the candidates provided in 'Profiles'. Do not add or rename candidates.\n"
                        "Use 'criteria' to compute weighted scores from: must_haves coverage, nice_to_haves coverage, "
                        "experience, education, projects, and apply penalties for disqualifiers in flags.disqualifiers_hit.\n"
                        "Normalization rules:\n"
                        "- Case-insensitive matching for skills.\n"
                        "- Handle minor typos and close variants (performing-arts context example: 'signing' ≈ 'singing').\n"
                        "- Plurals and common synonyms should match (e.g., 'actor' ≈ 'acting').\n"
                        "- Infer years from experience text when possible; otherwise 0.\n\n"
                        "Return ONLY JSON with two keys:\n"
                        "- ranked: array of objects with candidate_id, name, score_total (number), breakdown "
                        "(must_haves: matched[], missing[], score; nice_to_haves: matched[], score; "
                        "experience (number), education (number), projects (number); penalties: disqualifiers[], score_delta), "
                        "and rationale (string)\n"
                        "- top_n_ids: array of candidate_id strings for the top N\n"
                        "JSON only; no prose."
                    )
                },
                {
                    "role": "user",
                    "content": "$ f'''Criteria: {{ steps[0].input.criteria | tojson }}\n\nProfiles: {{ steps[0].input.extracted_profiles | tojson }}\n\nTop-N: {{ steps[0].input.N | tojson }}'''"
                },
            ],
            "unwrap": True,
            "save_as": "score_rank_json",  # JSON string from the model
        },
        {"return": {"score_rank_json": "$ steps[0].output"}},
    ],
}

score_task = client.tasks.create(agent_id=scorer.id, **task_score_def)
print("Task B ready:", score_task.id, score_task.name)

# ---------- Execution B ----------
exe_b = run_and_wait(score_task.id, {"criteria": criteria, "extracted_profiles": extracted_profiles, "N": N})
if exe_b.status != "succeeded":
    raise RuntimeError(f"Task B failed: {exe_b.output}")

out_b = exe_b.output
parsed_b = safe_json_loads(out_b) if isinstance(out_b, str) else (out_b or {})
score_rank_json = parsed_b.get("score_rank_json", parsed_b)
score_rank = safe_json_loads(score_rank_json) if isinstance(score_rank_json, str) else (score_rank_json or {})

ranked = score_rank.get("ranked", [])
top_n_ids = score_rank.get("top_n_ids", [])
print("Ranked:", json.dumps(ranked, indent=2, ensure_ascii=False))
print("Top-N IDs:", top_n_ids)

profiles_by_id = {p["candidate_id"]: p for p in extracted_profiles if "candidate_id" in p}
top_profiles = [profiles_by_id[cid] for cid in top_n_ids if cid in profiles_by_id]


Task B ready: 068ab752-830b-7bb9-8000-3b31ef121c7c score_rank_task
Execution: 068ab752-8831-7bae-8000-8d7eaa63f15c
Status: queued
Status: starting
Status: starting
Status: starting
Status: starting
Status: running
Status: succeeded
Final status: succeeded
Ranked: [
  {
    "candidate_id": "1",
    "name": "Alice Johnson",
    "score_total": 85,
    "breakdown": {
      "must_haves": {
        "matched": [
          "JavaScript",
          "React",
          "Node.js"
        ],
        "missing": [],
        "score": 30
      },
      "nice_to_haves": {
        "matched": [
          "GraphQL",
          "TypeScript"
        ],
        "score": 10
      },
      "experience": 20,
      "education": 15,
      "projects": 10,
      "penalties": {
        "disqualifiers": [],
        "score_delta": 0
      }
    },
    "rationale": "Alice Johnson covers all must-have skills and most nice-to-have skills. She has significant experience and relevant education, with multiple projects demonstr

In [94]:
# --- Cell 4: Task C — Draft Interview Questions (create & execute) ---

task_interview_def = {
    "name": "interview_task",
    "description": "Write tailored interview questions for the selected top candidates.",
    "input_schema": {
        "type": "object",
        "required": ["criteria", "top_profiles"],
        "properties": {
            "criteria": {"type": "object"},
            "top_profiles": {"type": "array", "items": {"type": "object"}}
        }
    },
    "main": [
        {
            "prompt": [
                {
                    "role": "system",
                    "content": (
                        "You are InterviewerAgent. For EACH candidate, write 6–10 tailored questions that reference their "
                        "background (skills, projects, experience) and probe the must-haves for the role. "
                        "Add a brief reason for each question in the field 'why_this_question'.\n\n"
                        "Return ONLY valid JSON with key 'interview_questions', which is an array of objects containing: "
                        "candidate_id, name, and questions (each question has fields 'question' and 'why_this_question'). "
                        "JSON only; no prose."
                    )
                },
                {
                    "role": "user",
                    "content": "$ f'''Criteria: {{ steps[0].input.criteria | tojson }}\n\nSelected candidates: {{ steps[0].input.top_profiles | tojson }}'''"
                },
            ],
            "unwrap": True,
            "save_as": "interview_json",  # JSON string
        },
        {"return": {"interview_json": "$ steps[0].output"}},
    ],
}

interview_task = client.tasks.create(agent_id=interviewer.id, **task_interview_def)
print("Task C ready:", interview_task.id, interview_task.name)

# ---------- Execution C ----------
exe_c = run_and_wait(interview_task.id, {"criteria": criteria, "top_profiles": top_profiles})
if exe_c.status != "succeeded":
    raise RuntimeError(f"Task C failed: {exe_c.output}")

out_c = exe_c.output
parsed_c = safe_json_loads(out_c) if isinstance(out_c, str) else (out_c or {})
interview_json = parsed_c.get("interview_json", parsed_c)
interview_parsed = safe_json_loads(interview_json) if isinstance(interview_json, str) else (interview_json or {})
questions = interview_parsed.get("interview_questions", [])
print("Interview questions:", json.dumps(questions, indent=2, ensure_ascii=False))


Task C ready: 068ab753-0af0-7699-8000-03b65aa23e0f interview_task
Execution: 068ab753-0ff8-7baa-8000-a65893669234
Status: queued
Status: starting
Status: starting
Status: starting
Status: starting
Status: starting
Status: starting
Status: starting
Status: starting
Status: succeeded
Final status: succeeded
Interview questions: [
  {
    "candidate_id": "1",
    "name": "Alice Johnson",
    "questions": [
      {
        "question": "Can you elaborate on your experience with cloud-based solutions, particularly in AWS, as mentioned in your resume?",
        "why_this_question": "The role requires strong experience with cloud platforms, and AWS is a key component. Understanding Alice's depth of experience will help assess her fit for the role."
      },
      {
        "question": "You have experience in leading a team of software developers. Can you describe a challenging project you led and how you managed it?",
        "why_this_question": "Leadership and project management skills are c